<h1><center>Laboratorio 3: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Josefa Anselmo
- Nombre de alumno 2: Tamara Carrasco


### **Link de repositorio de GitHub:** [TamoJosha](https://github.com/Tamaracarrasco/TamoJosha-Lab-de-Prog-MDS-Prim2025)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
%pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
  #  from google.colab import drive
   # drive.mount("/content/drive")
   # path = 'Dirección donde tiene los archivos en el Drive'
#except:
 #   print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
import plotly.io as pio

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle("online_retail_data.pickle")

df_retail.head(5)

In [ ]:
df_retail["Price"].describe()

In [ ]:
df_retail["Quantity"].describe()

In [ ]:
df_retail.info()

### 1. Función para explorar características [0.5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [0.3 puntos]
- Imprima un conteo de datos nulos por variable [0.2 puntos]

**Nota**: Para generar los gráficos no es obligatorio el uso de `plotly`, pero si es altamente recomendado. Pueden encontrar más información de esta librería en este [enlace](https://plotly.com/python/).

**Respuesta:**

In [ ]:
def explore_data(df, name, yscale="log"):
    """
    Función que retorna histogramas de las variables Price y Quantity de un dataframe que
    contiene esas columnas.

    Parámetros:
      df: DataFrame con columnas 'Price' y 'Quantity'
      name: nombre del dataframe. Debe ser un string
      yscale: modificamos la escala en el eje Y pues no se podía graficar "bien" con plotly.

    """
    # Se agrega esto porque tuve problemas en el renderizado de vscode
    pio.renderers.default = "notebook"
    
    # Imprimimos valores nulos
    print("Valores nulos por columna")
    print(df.isna().sum())

    # Histogramas con Plotly

    log = (yscale == "log") # Aquí redefinimos el suavizado que le vamos a aplicar al eje Y
                            # porque la escala actual no permite que se grafique correctamente.
    

    fig1 = px.histogram(
        df, x="Price", nbins=120,
        title=f"Distribución de Price del dataframe {name}",
        log_y=log
    ) # Aquí defino el plot para Prie

    fig1.show()

    fig2 = px.histogram(
        df, x="Quantity", nbins=120,
        title=f"Distribución de Quantity del dataframe {name}",
        log_y=log
    ) # Aquí defino el plot para Quantity
  
    fig2.show()


In [ ]:
# histogramas sin el suavizado logarítmico

explore_data(df_retail, "df_retail", yscale="no")

# Como se puede ver, es muy difícil identificar el comportamiento de esta variable con esta escala
# así que vamos a aplicar el log en la siguiente celda logrando así ver el comportamiento, en otra
# escala, pero de este modo si funciona en el fondo.

In [ ]:
# Compruebo resultados con suavizado logarítmico
explore_data(df_retail, "df_retail")

# Aquí se se puede ver que hay un comportamiento en la distribución de las variables.

### 2. Eliminando outliers [1.0 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [0.5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

**Hint:** tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Nota:** No modificar el método set_output de la clase IQR

**Respuesta:**

In [ ]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, cte=1.5):
    self.cte = cte # cte representa lambda que define el usuario
                   # por defecto es 1.5 (Ley de Tukey) y señala que ese sería el límite
                   # para considerar un punto como outlier en base a una convención "general"
                   # respecto de estos rangos pero que de todos modos uno podría definirlo
                   # si tiene nociones reales de como se mueven los rangos para considerar 
                   # los outliers.
    pass

  def fit(self, X, y=None): # Acá se aplica lo de la guía
    # hay que obtener solo columnas numéricas
    self.num_columns = X.select_dtypes(include='number').columns # lista de columnas del dataframe. 
                                                                 # Por como será el pipeline, 
                                                                 # deberian ser las columnas 
                                                                 # numéricas. Además, me aseguro
                                                                 # que solo sean numéricas.

    # No es necesario self.X = X ya que no se busca guardar el dataframe, solo calcular los límites
    self.q1 = X[self.num_columns].quantile(0.25) # Q1
    self.q3 = X[self.num_columns].quantile(0.75) # Q3
    self.ric = self.q3 - self.q1 # RIC

    self.lim_inf = self.q1 - self.ric*self.cte #lim. inferior
    self.lim_sup = self.q3 + self.ric*self.cte #lim. superior

    return self
    

  def transform(self, X): # de lo que entendí de la guía es que acá, X se transforma con lo que aprendió en fit (los límites)
                          # o sea acá se redefinen los outliers
    
    X_no_outlier = X.copy()
    
    mask = (( X_no_outlier[self.num_columns] < self.lim_inf) |
            ( X_no_outlier[self.num_columns] > self.lim_sup))

    # Enmascarar a NaN en lugar de eliminar el registro porque esto en el fondo
    # generaria que distintas variables tuvieran distinta cantidad de registros

    X_no_outlier.loc[:, self.num_columns] =  X_no_outlier[self.num_columns].mask(mask, other=np.nan)
    return  X_no_outlier

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [0.5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [0.1 puntos]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [0.1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [0.1 puntos]


**Hint:** El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.

**Nota:** Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Lambda de 1.5 (Ley de Tukey) - mantenemos el mismo valor de cte que en la clase para efectos de comparación y porqupe es el de la ley de Tukey por defecto

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ["Quantity", "Price"]
categorical_columns = ["Invoice", "StockCode" ,"Description", "InvoiceDate", "Customer ID" ,"Country"]

# Definicion del pipeline
numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=1.5))]) # el pipe espera una lista con tuplas (nombre, transformador)

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', "passthrough", categorical_columns) # dejo las categoricas con el pss porque aun no defino como las voy a tratar
                                        ],                                                   
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas') # esto es para asegurarnos que el output devuelva un dataframe y no un array

# Aplicamos ColumnTransformer a los datos

df_iqr = column_transformer.fit_transform(df_retail) # .fit() devuelve objeto entrenado no el dataframe transformado
                                                    # por eso se usa fit_trasnform que si devuelve el dataframe transformado

# Gráficos
print("Dataframe original sin suavizado logarítmico, cte=1.5")
explore_data(df_retail, "df_retail", yscale="no") # Plot inicial sin suavizado logarítmico
print("Dataframe original con suavizado logarítmico, cte=1.5")
explore_data(df_retail, "df_retail") # Plot original con suavizado logarítmico
print("Dataframe tras aplicar IQR sin suavizado logarítmico, cte=1.5")
explore_data(df_iqr, "df_iqr", yscale="no") # Plot IQR sin suavizado logarítmico
print("Dataframe tras aplicar IQR con suavizado logarítmico, cte=1.5")
explore_data(df_iqr, "df_iqr", yscale="log") # Plot IQR con suavizado logarítmico

In [ ]:
print("Valores nulos en el dataframe inicial, cte=1.5")
df_retail.isna().sum()

In [ ]:
print("Valores nulos en el dataframe tras aplicar IQR, cte=1.5")
df_iqr.isna().sum()

In [ ]:
# Lamda de 3 

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ["Quantity", "Price"]
categorical_columns = ["Invoice", "StockCode" ,"Description", "InvoiceDate", "Customer ID" ,"Country"]

# Definicion del pipeline
numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=3))]) # el pipe espera una lista con tuplas (nombre, transformador)

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', "passthrough", categorical_columns) # acá se pudo haber usado remainder si es que
                                        ],                                                    # no se especificaban las variables no numéricas 
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos

df_iqr = column_transformer.fit_transform(df_retail) # .fit() devuelve objeto entrenado no el dataframe trasnformado, por eso se usa fit_trasnform y no .fit

# Gráficos
print("Dataframe tras aplicar IQR sin suavizado logarítmico, cte=3")
explore_data(df_iqr, "df_iqr", yscale="no") # Plot IQR sin suavizado logarítmico
print("Dataframe tras aplicar IQR con suavizado logarítmico, cte=3")
explore_data(df_iqr, "df_iqr", yscale="log") # Plot IQR con suavizado logarítmico

In [ ]:
print("Valores nulos en el dataframe inicial, cte=3")
df_retail.isna().sum()

In [ ]:
print("Valores nulos en el dataframe tras aplicar IQR, cte=3")
df_iqr.isna().sum()

In [ ]:
# Lambda de 5 

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ["Quantity", "Price"]
categorical_columns = ["Invoice", "StockCode" ,"Description", "InvoiceDate", "Customer ID" ,"Country"]

# Definicion del pipeline
numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=5))]) # el pipe espera una lista con tuplas (nombre, transformador)

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', "passthrough", categorical_columns) # acá se pudo haber usado remainder si es que
                                        ],                                                    # no se especificaban las variables no numéricas 
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos

df_iqr = column_transformer.fit_transform(df_retail) # .fit() devuelve objeto entrenado no el dataframe transformado, 
                                                        #por eso se usa fit_trasnform y no .fit

# Gráficos
print("Dataframe tras aplicar IQR sin suavizado logarítmico, cte=5")
explore_data(df_iqr, "df_iqr" ,yscale="no") # Plot IQR sin suavizado logarítmico
print("Dataframe tras aplicar IQR con suavizado logarítmico, cte=5")
explore_data(df_iqr, "df_iqr" ,yscale="log") # Plot IQR con suavizado logarítmico

In [ ]:
print("Valores nulos en el dataframe inicial, cte=5")
df_retail.isna().sum()

In [ ]:
print("Valores nulos en el dataframe tras aplicar IQR, cte=5")
df_iqr.isna().sum()

**RESPUESTA**

Por lo que pudimos ver, al variar la cte entre 1.5, 3 y 5, pasa que mientras más grande es la cte, más amplio es el rango en que se permiten valores atípicos en las colas y, por lo mismo, menos datos se eliminan. O sea, una cte más chica es más estricta para sacar outliers que una cte más grande. Lo importante es que al momento de definir la cte esto no se haga al azar solo para ahorrarse lidiar con outliers o imputaciones, sino que se debe tener cierto nivel de conocimiento de las variables/distribución + conocimiento experto de la temática, y en base a eso definir una cte adecuada, que podamos argumentar y que haga sentido en el fondo.

También revisamos que existe la cte por defecto que deriva de la “Ley de Tukey”, la cual corresponde a 1.5 y se relaciona con la regla de “la pulgada”. En el fondo, esta regla establece una holgura de 1.5 veces el rango intercuartílico en las colas.

### 3. Agregando un imputer al pipeline [1.0 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [0.1 puntos]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [0.1 puntos]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [0.1 puntos]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [0.2 puntos]

**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer

# Defino los tipos de columnas
# Estos son los tipos de datos que tenemos en la base category, object, datetime64[ns] y float64 entonces definimos por separado la agrupación para que después se apliquen las imputaciones
# correspondientes

numeric_cols   = df_retail.select_dtypes(include='float64').columns.tolist()
categoric_cols = df_retail.select_dtypes(include=['object','category','datetime64[ns]']).columns.tolist()

# PARTE 1 

categoric_transformations = Pipeline(steps=[('mode_imputer', SimpleImputer(strategy='most_frequent'))]) # Aquí estamos definiendo el pipe que en el fondo imputará a las variables categóricas 
                                                                                                        # el valor más frecuente de la categoría
                                                                                                        # Si bien desde un punto de vista de "imputación" esto se podría hacer, no significa
                                                                                                        # que esta sea la forma correcta de imputar todas estas clases de datos. Estrictamente 
                                                                                                        # deberíamos analizar variable por variable e imputar según correspnda a la naturaleza 
                                                                                                        # de la misma y lo que se necesite encontrar por decir.
                                                                                                        # Acá no se explicitaba pero igual usamos SimpleImputer porque de esta forma salia bien.

# PARTE 2 
# Aquí piden agregar un paso de "mean_imputer" en el pipe de "numeric_transformations" por lo que se entiende que debemos mantener el mismo pipe que teniamos definido de antes y a este
# agregarle el paso que nos comentan. No se esta pidiendo que redefinamos o que creemos un nuevo pipe por decir, más solo agregar el paso.

numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=1.5)), ("mean_imputer", SimpleImputer(strategy="mean"))]) # El segundo parentesis, por decir, es el paso que se agrega acá porque 
                                                                                                                        # el resto venia de la definición inical de este pipe de una sección previa

# PARTE 3

ct_actualizado = ColumnTransformer( # con esto estoy creando el columntransformer para aplicar en la bbdd con el nombre de ct_actualizado
    transformers=[ #aqui defino las diferentes transformaciones que voy a a plicar sobre las diferentes columnas o tipos de datos en vd
        ('categoric_transformations', categoric_transformations, categoric_cols), # se compone como del nombre de la transformacion + el pipeline + las colunas donde lo aplicare que en este caso las definimos al inicio segun el tipo de datos que son
        ('numeric_transformations',   numeric_transformations,   numeric_cols)
    ],
    verbose_feature_names_out=False, # esto que tmb aplicamos antes es como para que las columnas no queden renombradas
    remainder='passthrough' # esto es como para que no tire error/levante alerta si hay columnas en el df que no se consideren dentro de las transformaciones, entonces así los deje tan cual
).set_output(transform='pandas') # por defecto ct me devuelve un array pro yo quiero el dataframe de pandas entonces lo fuerzo a que eso pase por decri

df_mean_imputer = ct_actualizado.fit_transform(df_retail) # con esto obtengo el df transformado pero las col no necesarimente están en el mismo orden
df_mean_imputer = df_mean_imputer[df_retail.columns] # Con esto puedo volverlas al mismo orden del inicio


# PARTE 4 
# ahora voy a comparar los datos de "df_mean_imputer" y "df_iqr" según lo definido al inicio en el "explore_data"

print("PARTE 4: explore_data(df_mean_imputer) vs explore_data(df_iqr)")
explore_data(df_mean_imputer, "df_mean_imputer")
explore_data(df_iqr, "df_iqr")

# PARTE 5 

# aqui redefino el pipe (en verdad cree uno nuevo por knn) con el nuevo imputador
numeric_transformations_knn = Pipeline([("iqr_transformer", IQR(cte=1.5)), ("knn_imputer", KNNImputer(n_neighbors=5, weights="uniform", metric="nan_euclidean"))])


ct_knn = ColumnTransformer( # Defino el columntransform
    transformers=[
        ('categoric_transformations', categoric_transformations, categoric_cols), # defino los cambios sobre los tipos de varibales
        ('numeric_transformations_knn',   numeric_transformations_knn,   numeric_cols),
    ],
    verbose_feature_names_out=False, # evito que cambie el nombre de la columna
    remainder='passthrough' # aqqui para que no modifique o tire error si hay variables que no tendran modificacion
).set_output(transform='pandas') # forzar al df de pandas en lugar del array

df_knn_imputer = ct_knn.fit_transform(df_retail) # aqui lo aplico ya a df_retail
df_knn_imputer = df_knn_imputer[df_retail.columns] # reordeno las columnas nuevamente


# PARTE 6
# aca comparo también los diferentes dataframes con las transformaciones de cada uno

print("PARTE 6: explore_data(df_knn_imputer) vs explore_data(df_iqr)")
explore_data(df_knn_imputer, "df_knn_imputer")
explore_data(df_iqr, "df_iqr")

# PARTE 7
# Primero tengo que definir la media y desviaion estandar de las varibales numericas, porque este será como mi 
# baseline para comparar que imputador es mejor

ref_mean = df_retail[numeric_cols].mean()
ref_std  = df_retail[numeric_cols].std(ddof=1)

# Hago como un cálculo de "distancias simples" pero en valor absoluto (|Δ media| + |Δ std|). 
# Mientras menor, mejor por que en el fondo implicaría que esta más cerca de lo "real".
dist_mean = (ref_mean - df_mean_imputer[numeric_cols].mean()).abs().sum() \
          + (ref_std  - df_mean_imputer[numeric_cols].std(ddof=1)).abs().sum()

dist_knn  = (ref_mean - df_knn_imputer[numeric_cols].mean()).abs().sum() \
          + (ref_std  - df_knn_imputer[numeric_cols].std(ddof=1)).abs().sum()

print("PARTE 7: KNN vs MEAN (comparación cuantitativa simple)")
print(f"Distancia media+std vs original -> MEAN: {dist_mean:.6f} | KNN: {dist_knn:.6f}")

# aca comparo los resultados
print("explore_data: df_knn_imputer vs df_mean_imputer")
explore_data(df_knn_imputer, "df_knn_imputer")
explore_data(df_mean_imputer, "df_mean_imputer")

# hago un if que me deje automaticamente elegir con cual nos vamos a quedar
if dist_knn <= dist_mean:
    ct_final = ct_knn
    df_imputer_final = df_knn_imputer
    metodo_elegido = "KNNImputer"
else:
    ct_final = ct_actualizado
    df_imputer_final = df_mean_imputer
    metodo_elegido = "SimpleImputer(mean)"

print(f"Método de imputación escogido: {metodo_elegido}")

### 4. Creación de nuevas features [2.0 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [1.0 puntos]

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def custom_features(dataframe_in):
    
    copia = dataframe_in.copy()

    copia = copia.sort_values(["Customer ID", "InvoiceDate"]) # se ordena porque será más útil para adelante

    # PARA LENGHT

    # Se obtienen columnas customer id, primera_visita y ultima_visita
    len = copia.groupby("Customer ID", observed=True)["InvoiceDate"].agg(primera_visita="min", ultima_visita="max").reset_index() # esto es un dataframe
   
    Lenght = (len["ultima_visita"] - len["primera_visita"]).dt.days # diferencia en días: esto no es un dataframe
    
    # PARA RECENCY
    Recency = np.abs((len["ultima_visita"] - pd.to_datetime("today")).dt.days) # valor absoluto: esto no es un dataframe

    # PARA MONETARY
    copia["dinero_gastado"] =  copia["Quantity"] * copia["Price"]
    
    Monetary = copia.groupby("Customer ID", observed=True)["dinero_gastado"].mean().round(2).fillna(0).reset_index(name="Monetary") 
    # promedio gastado por cliente: esto es un dataframe
    # acá me di cuenta que hay un producto con precio 0.0, luego la multiplicacion da nan
    
    # PARA FREQUENCY
    copia["fecha"] = copia["InvoiceDate"].dt.normalize() # convierte a midnight pero quedan los dd/mm/aaaa

    Frequency = copia.groupby("Customer ID", observed=True)["fecha"].nunique().reset_index(name="Frequency") 
    # dataframe con  columnas customer id, fecha y frequency

    # PARA PERIODICIDAD
    
    copia["IVT"] = copia.groupby("Customer ID", observed=True)["InvoiceDate"].diff().dt.days # obtenemos la diferencia de dias para cada cliente

    periodicidad = copia.groupby("Customer ID", observed=True)["IVT"].std().round(1).fillna(0).reset_index(name="Periodicity") # se aplica desviacion estandar y reseteamos index
    # periodicidad es dataframe con columnas customer id y periodicidad (periodicity)

    # si un cliente visita 1 solo dia, no hay std -> se tiene un nan
    # Si un cliente visita 2 dias, hay solo un intervalo -> no hay desviación estandar -> arroja nan
    
    dataframe_out = pd.DataFrame({
        "Customer ID": len["Customer ID"],
        "Lenght": Lenght,
        "Recency": Recency,
        "Monetary": Monetary["Monetary"],
        "Frequency": Frequency["Frequency"] ,
        "Periodicity": periodicidad["Periodicity"]

    })
    return dataframe_out

#### 4.2 Agregando las custom features [1.0 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [0.1 puntos]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [0.1 puntos]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [0.5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [0.3 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
custom_features_transformer = FunctionTransformer(custom_features) # uso del hint que pusieron

retail_pipe = Pipeline(steps=[
                                ("col_transformer", ct_knn), 
                                ("custom_features", custom_features_transformer)    
                            ])                                                      # se escogió el col_trans de knn

df_custom = retail_pipe.fit_transform(df_retail) # acá, se realizan las imputaciones correspondientes con knn

In [ ]:
# Definición de una nueva funcion explore_data

def explore_custom(df, name, yscale="log"):
    """ Función que recibe un dataframe, el cual debe contener las columnas
    "Lenght", "Recency", "Monetary", "Frequency", "Periodicity" y 
    retorna un histograma de cada variable.

    Parámetros:
    --------------
    df: dataframe a tratar.

    name: (str) Nombre del dataframe.

    yscale: (str) Modifica la escala del eje y porque a veces no
                  se puede graficar bien con plotly debido a la cantidad 
                  de los datos.


    """
    
    pio.renderers.default = "notebook"

    # Impresión de valores nulos por columna
    print("Valores nulos por columna")
    print(df.isna().sum()) # debería dar 0 porque hubo imputación en todas las variables.

    # Histogramas

    log = (yscale =="log")

    # Lenght

    fig1 = px.histogram(
        df, x = "Lenght", nbins=120,
        title=f"Distribución de Length del dataframe {name}",
        log_y = log
    )

    fig1.show()

    # Recency

    fig2 = px.histogram(
        df, x = "Recency", nbins=120,
        title=f"Distribución de Recency del dataframe {name}",
        log_y = log
    )

    fig2.show()

    # Monetary

    fig3 = px.histogram(
        df, x = "Monetary", nbins=120,
        title=f"Distribución de Monetary del dataframe {name}",
        log_y = log
    )

    fig3.show()
    
    # Frequency

    fig4 = px.histogram(
        df, x = "Monetary", nbins=120,
        title=f"Distribución de Frequency del dataframe {name}",
        log_y = log
    )

    fig4.show()

    # Periodicidad

    
    fig4 = px.histogram(
        df, x = "Periodicity", nbins=120,
        title=f"Distribución de Periodicity del dataframe {name}",
        log_y = log
    )

    fig4.show()


In [ ]:
explore_custom(df_custom, "df_custom", yscale="no")

### 5. MinMax Scaler [1.0 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [0.5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self,X):
        #Agregar código aquí
        pass

    def transform(self,X):
        #Agregar código aquí
        pass

    def set_output(self,transform='default'):
        #No modificar este método
        return self



#### 5.2 Incorporando MinMax al pipeline [0.5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [0.1 puntos]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables.  [0.2 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

### 6. Pregunta teórica [0.5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Bueno Mr. Cheems, lo que ocurre es que cuando hablamos de *Feature Engineering* nos referimos al proceso de tomar los datos en bruto y transformarlos en variables que sean más claras, útiles y representativas para que un modelo de *Machine Learning* pueda aprender mejor los patrones. La verdad es que no importa tanto si el modelo es complejo o simple, lo que marca la diferencia es la calidad de las variables. O sea, un modelo sencillo, pero con buenas variables puede funcionar mucho mejor que uno complicado con variables mal hechas.

Por lo mismo, aparecen los *pipelines*, que son súper importantes porque permiten ordenar todos los pasos de transformación de los datos antes de entrenar el modelo. En vez de ir haciendo cada cosa a mano una y otra vez, el pipeline asegura que procesos como rellenar valores faltantes, escalar, reducir dimensiones o seleccionar variables se hagan siempre en el mismo orden y de la misma manera. Eso ayuda a evitar errores y da más seguridad en lo que estamos haciendo.

Gracias a los pipelines podemos tener *reproducibilidad* en el código porque cada vez que se corre, los pasos se repiten exactamente igual, sin sorpresas. También ayuda a evitar el *data leakage*, ya que las transformaciones se ajustan solo con los datos de entrenamiento y después se aplican al conjunto de prueba o validación, respetando la separación entre ellos (o sea, los conjuntos no se “conocen” de antes).

Otro punto importante, e igual derivado de lo que explicamos antes, es que los pipelines hacen el trabajo mucho más *rápido y automático* porque basta con llamarlos una vez y todos los pasos se ejecutan sin tener que repetir cada transformación manualmente. Esto además hace que sea más *fácil probar cambios*, porque si quiero reemplazar un paso, lo cambio en el pipeline y se aplica en todo el flujo, sin tener que ir corrigiendo celda por celda.

Y, por último, los pipelines también funcionan muy bien con cosas como la *validación cruzada* y la *búsqueda de hiperparámetros*. Esto básicamente significa que, cuando queremos probar distintas configuraciones de un modelo (por ejemplo, distintos parámetros o combinaciones de transformaciones), el pipeline se encarga de que todo el proceso se ejecute de manera ordenada desde el principio hasta el final. Así, no solo se entrena el modelo, sino que también se aplican siempre las mismas transformaciones a los datos en cada prueba, lo que asegura que el preprocesamiento y el modelado estén realmente conectados y trabajen como un todo.

Entonces, a modo de resumen, los pipelines son súper útiles porque hacen que todo el proceso de preparar los datos y entrenar modelos sea más ordenado, seguro y eficiente. Ayudan a evitar errores, facilitan la experimentación y aseguran que todo el trabajo que hacemos con los datos se mantenga consistente y reproducible, así que es clave usarlo ademas de ser una buena práctica en ciencia de datos.